In [ ]:
from dataclasses import dataclass

@dataclass
class Base:
    val: int = 0

@dataclass
class Derived(Base):
    val: int = 2

In [ ]:
A = Derived()
print(A.val)

In [ ]:
import jax.numpy as jnp
a = jnp.array([0])
a.shape